# Sklearn Example: LogisticRegression and Iris

In this example we will train as LogisticRegression model on the iris dataset, convert it to ONNX format, simulate historical treatment data, then estimate the treatment effects using our IV estimation method.

### 1. Train and Convert

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

In this example, X consists of 4 continuous variables

In [3]:
print("Dtype:", X.dtype, "\nShape:", X.shape)

Dtype: float64 
Shape: (150, 4)


Our estimation method relies on binary treatment assignment, so here let's assume that iris classifications of 1 or 2 indicate treatment recommendation.

In [4]:
y[y>0] = 1

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
clr = LogisticRegression()
clr.fit(X_train, y_train)

LogisticRegression()

In [7]:
clr.predict_proba(X_test)[:5]

array([[4.05205203e-04, 9.99594795e-01],
       [9.47539986e-01, 5.24600143e-02],
       [2.10102151e-04, 9.99789898e-01],
       [1.35983036e-01, 8.64016964e-01],
       [9.74173988e-01, 2.58260118e-02]])

In [8]:
import pickle 

# Save model
with open(f"models/iris_logreg.pickle", "wb") as f:
    pickle.dump(clr, f)

Now let's load back in the model and convert to ONNX. This step is necessary because our QPS estimation procedure only takes ONNX models for inference.

In [9]:
model = pickle.load(open(f"models/iris_logreg.pickle", 'rb'))
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

The mlisne package provides an ONNX conversion wrapper function that requires a dummy input and framework string to process datatypes and run the correct conversion function.

In [10]:
from mlisne.helpers import convert_to_onnx

X_dummy = X[0,:]
filename = "models/iris_logreg.onnx"

convert_to_onnx(model = model, dummy_input = X_dummy, path = filename, framework = "sklearn")

True

The wrapper function will also pass on any keyword arguments to the downstream conversion function

In [11]:
convert_to_onnx(model=model, dummy_input=X_dummy, path=filename, framework="sklearn", target_opset=12, 
                doc_string="Sklearn LogisticRegression model trained on iris dataset")

True

### 2. Simulate historical treatment data

Let's simulate treatment and non-treatment outcomes for each observation i with the following structural equations:
\begin{align}
Y_{0i} &= \mathcal{N}(2,2)x_{1i} + \mathcal{N}(3,1)x_{2i} + \mathcal{N}(0,3) \\
Y_{1i} &= Y_{0i} + \mathcal{N}(5,1) + \mathcal{N}(0,1)
\end{align}

Where $x_{1}$ and $x_{2}$ refer to the first and second variables of input X. 

In [12]:
# Generate coefficients for observations
b1 = np.random.normal(2,2,len(X))
b2 = np.random.normal(3,1,len(X))
e0 = np.random.normal(0,3,len(X))
treatment_effect = np.random.normal(5,1, len(X))
e1 = np.random.normal(0,1,len(X))

In [13]:
Y0 = b1 * X[:,0] + b2 * X[:,1] + e0
Y1 = Y0 + treatment_effect + e1

Generate treatment recommendation probabilities using our trained model. Let's assume that the treatment is assigned when the model recommends assignment 75% of the time (no-defiers assumption).

In [14]:
# Create treatment recommendation probabilities
Z_probs = clr.predict_proba(X)[:,1]
Z_probs[:5]

array([0.019318  , 0.02625827, 0.01666684, 0.02640822, 0.01710709])

In [15]:
# Draw treatment recommendations
recommend_prob = np.random.uniform(size=len(X))
Z = (recommend_prob <= Z_probs).astype(int)
Z[:5]

array([0, 0, 0, 0, 1])

In [16]:
# Realized assignment follows recommended assignment 75% of the time
treatment_prob = np.random.uniform(size=len(X))
D = []
for i in range(len(treatment_prob)):
    if Z[i] == 1:
        if treatment_prob[i] >= 0.75:
            D.append(0)
        else:
            D.append(1)
    else:
        D.append(Z[i])

In [17]:
sim_data = np.stack((Y0, Y1, Z, D), axis=1)
sim_data.shape

(150, 4)

In [18]:
full_data = np.concatenate((sim_data, X), axis=1)
full_data.shape

(150, 8)

In [19]:
# Save to csv
cols = ['Y0', 'Y1', 'Z', 'D', 'X1', 'X2', 'X3', 'X4']
df = pd.DataFrame(data = full_data, columns=cols)
df['Y'] = df['Y1']
df.loc[df['D']==0, 'Y'] = df.loc[df['D']==0, 'Y0']
df.to_csv("data/iris_data.csv", index=False)

In [20]:
# Treatment effects 
ate = (df.Y1 - df.Y0).mean()
atet = (df.loc[df['D'] == 1, 'Y1'] - df.loc[df['D'] == 1, 'Y0']).mean()
late = (df.loc[(df['D'] == df['Z']), 'Y1'] - df.loc[(df['D'] == df['Z']), 'Y0']).mean()
print(f"ATE: {ate}")
print(f"ATET: {atet}")
print(f"LATE: {late}")

ATE: 4.938243400674545
ATET: 4.921907342614233
LATE: 4.88420353014053


### 3. QPS Estimation

Let's load the simulated historical data into our EstimatorDataset class

In [21]:
from mlisne.dataset import EstimatorDataset
from mlisne.qps import estimate_qps

The data can be loaded upon initialization with optional variable indices, as well as indiviudally post-initialization. 

Variables without indices will be inferred to be ordered as follows: [Y, Z, D, X_d, X_c], where X_d is the set of discrete input variables, and X_c is the set of continuous input variables. 

For this example, our input data is all continuous.

In [22]:
iv_data = EstimatorDataset(data=df.drop(['Y0', 'Y1'], axis=1), Z=0, D=1, X_c = range(2,6))
iv_data.Y[:5] # Y inferred as last column

array([13.09555636, 17.13767785, 16.67381797, 18.46146531, 32.84339348])

In [23]:
# Overwrite data post-initialization
iv_data.load_data(Y = df['Y'], D = df['D'])
iv_data.Y[:5]

array([13.09555636, 17.13767785, 16.67381797, 18.46146531, 32.84339348])

In [24]:
# Discrete data is empty
iv_data.X_d

array([], shape=(150, 0), dtype=float64)

We can now estimate QPS using our IV dataset and converted ONNX model. 

**Important:** to allow for cross-framework compatibility, our qps estimation function only works with ONNX models that take single inputs or separate continuous/discrete inputs with specified node names, and outputs with name "output_probability". The convert_to_onnx function will output with those settings automatically, but please keep this in mind if you plan on using an externally generated ONNX model.

In [25]:
qps = estimate_qps(iv_data, S=100, delta=0.8, ML_onnx=filename)
qps[:5]

array([0.03226412, 0.04161337, 0.03315811, 0.04031405, 0.0291432 ])

The converted ONNX model sets expected input dtypes according to the dummy input, but you can also pass the continuous/discrete datatypes explicitly as a tuple to coerce the input data to a specific type.

In [26]:
iv_data.X_c.dtype

dtype('float64')

In [27]:
qps = estimate_qps(iv_data, S=100, delta=0.8, ML_onnx=filename, types=(np.float64, np.float64))
qps[:5]

array([0.03820143, 0.04776963, 0.03259791, 0.04177242, 0.02697258])

### 4. Treatment effect estimation

We can now estimate LATE using our IV approach

In [28]:
from mlisne.estimator import TreatmentEffectsEstimator

est = TreatmentEffectsEstimator()

In [29]:
est.fit(iv_data, qps)

We will fit on 150 values out of 150 from the dataset for which the QPS estimation is nondegenerate.


In [30]:
b0, b1, b2 = est.coef
print("Treatment effect estimate:", b1)
print("True LATE:", late)

Treatment effect estimate: 13.865922193168899
True LATE: 4.88420353014053


The fitted estimator also provides other estimation diagnostics

In [31]:
print(est)

+-----------+-----------+----------+---------+---------+----------+----------+
|           | Parameter | Std. Err |  T-Stat | P-Value | Lower CI | Upper CI |
+-----------+-----------+----------+---------+---------+----------+----------+
|   const   |   18.782  |  1.3041  | 14.4028 |   0.0   | 16.2051  |  21.359  |
| Treatment |  13.8659  |  2.6641  |  5.2047 |   0.0   |  8.6013  | 19.1305  |
|    QPS    |  -4.0282  |  2.3783  | -1.6937 |  0.0924 | -8.7281  |  0.6717  |
+-----------+-----------+----------+---------+---------+----------+----------+


In [32]:
# First stage details
fs = est.firststage

print("First stage coefficients:", fs['coef'])
print("First stage p-values:", fs['p'])
print("First stage r2:", fs['r2'])

First stage coefficients: [ 0.0141526  0.9069103 -0.1491705]
First stage p-values: [0.00389134 0.         0.02094199]
First stage r2: 0.5456202817448683


In [33]:
est.firststage_summary()

+-------------------+-----------+----------+---------+---------+----------+----------+
|                   | Parameter | Std. Err |  T-Stat | P-Value | Lower CI | Upper CI |
+-------------------+-----------+----------+---------+---------+----------+----------+
|       const       |   0.0142  |  0.0048  |  2.9333 |  0.0039 |  0.0046  |  0.0211  |
| ML Recommendation |   0.9069  |  0.057   | 15.9122 |   0.0   |  0.7943  |  0.9894  |
|        QPS        |  -0.1492  |  0.0639  | -2.3342 |  0.0209 | -0.2755  | -0.0567  |
+-------------------+-----------+----------+---------+---------+----------+----------+
